# CNN training loop - network programming

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision.transforms import transforms

torch.set_printoptions(linewidth=120)

In [14]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [15]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        t = t

        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride = 2)

        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = t.reshape(-1, 12*4*4)
        t = self.fc1(t)
        t = F.relu(t)
        
        t = self.fc2(t)
        t = F.relu(t)

        t = self.out(t)

        return t

In [16]:
train_set = torchvision.datasets.FashionMNIST(root='/.data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))



# training with a single batch

In [17]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

batch = next(iter(train_loader))
images, labels = batch

preds = network(images)
loss = F.cross_entropy(preds, labels)

loss.backward()
optimizer.step()

print(f'loss1: {loss.item()}')
preds = network(images)
loss = F.cross_entropy(preds, labels)
print(f'loss2: {loss.item()}')

loss1: 2.296238422393799
loss2: 2.2761032581329346


# Training all batches. single epoch

In [18]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

total_loss = 0
total_correct = 0

for batch in train_loader:
    images, labels = batch

    preds = network(images)
    loss = F.cross_entropy(preds, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    total_correct += get_num_correct(preds, labels)

    print(f'epoch: {0}, total_correct: {total_correct}, loss: {total_loss}')


epoch: 0, total_correct: 8, loss: 2.3030991554260254
epoch: 0, total_correct: 19, loss: 4.615457057952881
epoch: 0, total_correct: 33, loss: 6.924859523773193
epoch: 0, total_correct: 55, loss: 9.20805549621582
epoch: 0, total_correct: 79, loss: 11.487581968307495
epoch: 0, total_correct: 110, loss: 13.70854139328003
epoch: 0, total_correct: 139, loss: 15.8880033493042
epoch: 0, total_correct: 180, loss: 17.84353768825531
epoch: 0, total_correct: 211, loss: 19.6875981092453
epoch: 0, total_correct: 249, loss: 21.55051052570343
epoch: 0, total_correct: 288, loss: 23.215041041374207
epoch: 0, total_correct: 331, loss: 24.834601998329163
epoch: 0, total_correct: 385, loss: 26.25756275653839
epoch: 0, total_correct: 426, loss: 27.714205861091614
epoch: 0, total_correct: 473, loss: 29.038708329200745
epoch: 0, total_correct: 534, loss: 30.137611865997314
epoch: 0, total_correct: 597, loss: 31.2419216632843
epoch: 0, total_correct: 652, loss: 32.37438440322876
epoch: 0, total_correct: 707, l

In [19]:
total_correct / len(train_set)

0.7986833333333333

# training with multi epoch

In [20]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)


for epoch in range(5):

    total_loss = 0
    total_correct = 0

    for batch in train_loader:
        images, labels = batch

        preds = network(images)
        loss = F.cross_entropy(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

        print(f'epoch: {epoch}, total_correct: {total_correct}, loss: {total_loss}')


epoch: 0, total_correct: 16, loss: 2.3012144565582275
epoch: 0, total_correct: 36, loss: 4.620950222015381
epoch: 0, total_correct: 44, loss: 6.920759916305542
epoch: 0, total_correct: 48, loss: 9.192548036575317
epoch: 0, total_correct: 57, loss: 11.41904330253601
epoch: 0, total_correct: 72, loss: 13.569576978683472
epoch: 0, total_correct: 101, loss: 15.598703384399414
epoch: 0, total_correct: 144, loss: 17.44759702682495
epoch: 0, total_correct: 190, loss: 19.045411348342896
epoch: 0, total_correct: 228, loss: 20.84282386302948
epoch: 0, total_correct: 254, loss: 22.492620825767517
epoch: 0, total_correct: 278, loss: 24.14717721939087
epoch: 0, total_correct: 323, loss: 25.58428716659546
epoch: 0, total_correct: 361, loss: 27.20201289653778
epoch: 0, total_correct: 397, loss: 28.677056193351746
epoch: 0, total_correct: 440, loss: 30.062125325202942
epoch: 0, total_correct: 483, loss: 31.559866666793823
epoch: 0, total_correct: 531, loss: 32.83318912982941
epoch: 0, total_correct: 5

In [21]:
total_correct / len(train_set)

0.8794